In [2]:
import cupy as cp

# Flush the CuPy memory cache
cp.get_default_memory_pool().free_all_blocks()
import cupy as cp

In [3]:
# Check memory pool information
memory_pool = cp.get_default_memory_pool()
print("Allocated memory:", memory_pool.total_bytes() / (1024 **3 ), "GB")

Allocated memory: 0.0 GB


In [4]:
import cupy as cp  # CuPy for GPU acceleration
import cupyx.scipy.sparse as cpx_sparse
from cupyx.scipy.sparse.linalg import gmres, LinearOperator
import pyvista as pv
import numpy as np  # NumPy for handling some CPU-based operations
from tqdm import tqdm
from scipy.sparse.linalg import bicgstab  # Fallback to CPU-based solver
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [5]:

# Material Properties
cohesion = 50  # Cohesion in Pascals
friction_angle = 25  # Friction angle in degrees
density = 1500  # Density in kg/m³
E = 1e7  # Young's modulus in Pascals
nu = 0.4  # Poisson's ratio

In [6]:
# Load the 3D Surface Mesh
def load_surface_mesh(filepath):
    print("Loading surface mesh...")
    mesh = pv.read(filepath)
    print("Surface mesh loaded successfully.")
    return mesh

filepath = r"H:\Others\Backup from 1TB Anvita HDD\Orissa\Data\4_Tiringpahada\Slope_Stability_WS\code_ws\07-08-2024\2D_Meshes\Filtered_test6_mesh.vtk"
mesh = load_surface_mesh(filepath)

Loading surface mesh...
Surface mesh loaded successfully.


In [7]:
# Assign Material Properties
def assign_material_properties_3d(mesh, cohesion, friction_angle, density, E, nu):
    print("Assigning material properties for 3D analysis...")
    mesh['cohesion'] = np.full(mesh.n_cells, cohesion)
    mesh['friction_angle'] = np.full(mesh.n_cells, friction_angle)
    mesh['density'] = np.full(mesh.n_cells, density)
    mesh['E'] = np.full(mesh.n_cells, E)
    mesh['nu'] = np.full(mesh.n_cells, nu)
    print("Material properties assigned.")

assign_material_properties_3d(mesh, cohesion, friction_angle, density, E, nu)

Assigning material properties for 3D analysis...
Material properties assigned.


In [8]:

# Compute Jacobian for 3D Surface Mesh
def compute_jacobian_3d(element_nodes, dN_dxi):
    """
    Compute the Jacobian for a 3D surface element (triangular or quadrilateral).
    
    Parameters:
    - element_nodes: Nx3 CuPy array representing the (x, y, z) coordinates of the element's nodes.
    - dN_dxi: Nx2 CuPy array representing the shape function derivatives with respect to the local coordinates.
    
    Returns:
    - J: 2x3 CuPy array representing the Jacobian matrix (mapping local to global coordinates).
    """
    # Project to 3D by using x, y, z coordinates
    J = cp.zeros((2, 3))  # 2x3 Jacobian matrix for surface element in 3D

    for i in range(3):  # Loop over the nodes of a triangular element
        J += cp.outer(dN_dxi[i], element_nodes[i])

    return J

# Compute B Matrix for 3D Surface Mesh
def compute_B_matrix_3d(J_inv, dN_dxi):
    B = cp.zeros((3, 9))  # 3 strains (xx, yy, xy) and 3 displacements per node, 3 nodes * 3 = 9
    for i in range(3):
        dN_dx = cp.dot(J_inv, dN_dxi[i])
        B[0, i*3] = dN_dx[0]  # ε_xx
        B[1, i*3+1] = dN_dx[1]  # ε_yy
        B[2, i*3] = dN_dx[1]  # ε_xy
        B[2, i*3+1] = dN_dx[0]
    return B

# Compute C Matrix for 3D
def compute_C_matrix_3d(E, nu):
    C = cp.zeros((3, 3))  # 3x3 matrix for 2D stress-strain relationship in a 3D context
    factor = E / (1 - nu**2)
    C[0, 0] = C[1, 1] = factor
    C[0, 1] = C[1, 0] = factor * nu
    C[2, 2] = E / (2 * (1 + nu))
    return C

# Compute Element Stiffness for 3D Surface Mesh
def compute_element_stiffness_3d(mesh, E, nu, nodes):
    dN_dxi = cp.array([
        [-1, -1],
        [1, 0],
        [0, 1]
    ])
    
    element_nodes = cp.array(mesh.points[nodes])  # Ensure this is CuPy
    J = compute_jacobian_3d(element_nodes, dN_dxi)
    det_J = cp.linalg.det(cp.dot(J, J.T))  # Determinant of the Jacobian
    
    if cp.abs(det_J) < 1e-12:
        print("Warning: Degenerate element detected. Skipping element.")
        return None
    
    J_inv = cp.linalg.inv(J[:2, :2])  # Inverse of the 2x2 Jacobian for in-plane coordinates
    B = compute_B_matrix_3d(J_inv, dN_dxi)
    C = compute_C_matrix_3d(E, nu)
    K_elem = cp.dot(B.T, cp.dot(C, B)) * det_J
    return K_elem

# Assemble Global Stiffness Matrix for 3D Surface Mesh
def assemble_global_stiffness_efficient_3d(K_global, K_elem, element):
    num_dofs_per_node = 3  # For 3D surface mesh, 3 DOFs per node
    num_nodes = len(element)
    global_indices = cp.array([int(node * num_dofs_per_node) for node in element], dtype=cp.int32)
    
    data = []
    rows = []
    cols = []
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            global_i = global_indices[i]
            global_j = global_indices[j]
            
            for k in range(num_dofs_per_node):
                for l in range(num_dofs_per_node):
                    value = K_elem[i*num_dofs_per_node+k, j*num_dofs_per_node+l]
                    data.append(value)
                    rows.append(global_i+k)
                    cols.append(global_j+l)
    
    data = cp.array(data)
    rows = cp.array(rows)
    cols = cp.array(cols)
    
    # Create a COO matrix and add it to the global stiffness matrix
    K_global += cpx_sparse.coo_matrix((data, (rows, cols)), shape=K_global.shape).tocsr()
    return K_global

# Compute Global Stiffness Matrix for 3D Surface Mesh
def compute_global_stiffness_matrix_3d(mesh, E, nu):
    print("Computing global stiffness matrix for 3D surface mesh...")
    K_global = cpx_sparse.coo_matrix((mesh.n_points * 3, mesh.n_points * 3))  # Start with COO format
    
    for i in tqdm(range(mesh.n_cells)):
        cell = mesh.get_cell(i)
        nodes = np.array(cell.point_ids).astype(int)  # Use NumPy array for indexing
        K_elem = compute_element_stiffness_3d(mesh, E, nu, nodes)
        
        if K_elem is not None:
            K_global = assemble_global_stiffness_efficient_3d(K_global, K_elem, nodes)
    
    print("Global stiffness matrix computed.")
    return K_global.tocsr()  # Convert to CSR format after assembly



K_global = compute_global_stiffness_matrix_3d(mesh, E, nu)

Computing global stiffness matrix for 3D surface mesh...


100%|██████████████████████████████████████████████████████████████████████████████| 5888/5888 [01:58<00:00, 49.49it/s]

Global stiffness matrix computed.


In [9]:

water_table_depth = 593  # Depth of water table in meters
pore_pressure = 1e2  # Pore pressure in Pascals
surcharge_load = 1000  # Surcharge load in Pascals
seismic_coefficient = 0.1  # Seismic load factor
# Define shapefile path
shapefile_path = r"H:\Others\Backup from 1TB Anvita HDD\Orissa\Data\4_Tiringpahada\Slope_Stability_WS\code_ws\07-08-2024\Test_dems\surcharge load shapes.shp"


In [10]:


# Apply Gravity Load
def apply_gravity_load_3d(mesh, density):
    print("Applying gravity load...")
    F_global = cp.zeros(mesh.n_points * 3)  # 3 DOFs per node in 3D
    F_global[2::3] -= density * 9.81  # Apply gravity in the negative z-direction (index 2)
    print("Gravity load applied.")
    return F_global

def apply_pore_pressure_3d(mesh, water_table_elevation, pore_pressure):
    """
    Apply predefined pore water pressure normal to each cell face in a mesh.

    Parameters:
    mesh (pyvista.PolyData): The mesh to which the load is applied.
    water_table_elevation (float): The elevation of the water table (e.g., in meters above sea level).
    pore_pressure (float): The predefined pore water pressure (e.g., in Pascals).

    Returns:
    cp.ndarray: The global force vector after applying the pore water pressure.
    """
    print("Applying predefined pore water pressure normal to each cell face...")

    # Compute cell centers, normals, and areas
    cell_centers = cp.array(mesh.cell_centers().points)
    cell_normals = cp.array(mesh.cell_normals)
    cell_areas = cp.array(mesh.compute_cell_sizes(length=False, area=True, volume=False)['Area'])

    # Initialize the global force vector (3 DOFs per node in 3D)
    F_global = cp.zeros(mesh.n_points * 3)

    # Iterate through each cell
    for i in range(mesh.n_cells):
        center = cell_centers[i]
        normal = cell_normals[i]
        area = cell_areas[i]

        # Check if the cell center elevation is below the water table elevation
        if center[2] < water_table_elevation:
            # Compute the force vector normal to the cell surface using the predefined pore pressure
            force_vector = pore_pressure * area * normal

            # Apply force vector to the nodes of the cell
            cell_point_ids = mesh.get_cell(i).point_ids
            num_points_in_cell = len(cell_point_ids)

            for point_id in cell_point_ids:
                F_global[point_id * 3: point_id * 3 + 3] += force_vector / num_points_in_cell  # Distribute force evenly among cell nodes

    print("Predefined pore water pressure applied normal to each cell face.")
    return F_global


# Apply Surcharge Load
def apply_surcharge_load_3d(mesh, shapefile_path, surcharge_load, height_tolerance=None):
    """
    Apply a surcharge load to mesh nodes within the polygons defined in a shapefile.
    
    Parameters:
    mesh (pyvista.PolyData): The mesh to which the load is applied.
    shapefile_path (str): Path to the shapefile containing the polygon(s) for load application.
    surcharge_load (float): The magnitude of the surcharge load to be applied.
    height_tolerance (float or None): The tolerance for the height difference when applying the load.
    
    Returns:
    cp.ndarray: The global force vector after applying the load.
    """
    
    print("Reading shapefile...")
    polygons = gpd.read_file(shapefile_path)
    
    if polygons.empty:
        print("Shapefile is empty or not loaded correctly.")
        return cp.zeros(mesh.n_points * 3)

    print("Applying surcharge load to areas defined by the polygon...")
    
    # Convert mesh points to a CuPy array for processing
    points = cp.array(mesh.points)
    F_global = cp.zeros(mesh.n_points * 3)  # Initialize the global force vector (3 DOFs per node in 3D)

    # Convert points to NumPy for Shapely compatibility
    points_np = points.get()

    # Loop through all polygons in the shapefile
    for polygon in polygons['geometry']:
        nodes_within_polygon = []

        # Iterate over each point in the mesh
        for node_index, point in enumerate(points_np):
            # Convert mesh point to a Shapely Point (using only X and Y for 2D polygon check)
            shapely_point = Point(point[0], point[1])

            # Check if the point is within the polygon
            if polygon.contains(shapely_point):
                print(f"Point {point} is within polygon.")
                if height_tolerance is None or abs(point[2] - polygon.bounds[2]) <= height_tolerance:
                    nodes_within_polygon.append(node_index)

        print(f"Polygon has {len(nodes_within_polygon)} nodes within height tolerance.")

        # Convert the list of nodes to a CuPy array of integers
        if len(nodes_within_polygon) > 0:
            nodes_within_polygon = cp.array(nodes_within_polygon).astype(int)

            # Apply the surcharge load to these nodes in the negative z-direction (index 2)
            F_global[nodes_within_polygon * 3 + 2] -= surcharge_load
        else:
            print("No nodes found within the polygon and height tolerance.")

    print("Surcharge load applied to the specified polygon areas.")

    return F_global


# Apply Seismic Load
def apply_seismic_load_3d(mesh, seismic_coefficient, direction='random'):
    print("Applying seismic load...")
    points = cp.array(mesh.points)
    F_global = cp.zeros(mesh.n_points * 3)  # 3 DOFs per node in 3D

    # Compute seismic forces based on chosen direction
    if direction == 'uniform':
        seismic_force_x = seismic_coefficient * points[:, 2]  # Proportional to height for x-direction
        seismic_force_y = seismic_coefficient * points[:, 2]  # Proportional to height for y-direction
    elif direction == 'gradient_x':
        seismic_force_x = seismic_coefficient * points[:, 2] * (points[:, 0] / cp.max(points[:, 0]))
        seismic_force_y = seismic_coefficient * points[:, 2] * (points[:, 0] / cp.max(points[:, 0]))
    elif direction == 'gradient_y':
        seismic_force_x = seismic_coefficient * points[:, 2] * (points[:, 1] / cp.max(points[:, 1]))
        seismic_force_y = seismic_coefficient * points[:, 2] * (points[:, 1] / cp.max(points[:, 1]))
    elif direction == 'random':
        seismic_force_x = seismic_coefficient * points[:, 2] * cp.random.uniform(0.5, 1.5, size=points[:, 2].shape)
        seismic_force_y = seismic_coefficient * points[:, 2] * cp.random.uniform(0.5, 1.5, size=points[:, 2].shape)

    # Apply computed forces to the global force vector
    F_global[::3] += seismic_force_x  # Apply in x-direction (index 0)
    F_global[1::3] += seismic_force_y  # Apply in y-direction (index 1)
    
    print("Seismic load applied with direction variation.")
    return F_global

# Apply All Loads for 3D Analysis
def apply_loads_3d(mesh, density, water_table_depth, pore_pressure, surcharge_load, seismic_coefficient):
    print("Applying all loads...")
    F_global = cp.zeros(mesh.n_points * 3)  # 3 DOFs per node in 3D
    F_global += apply_gravity_load_3d(mesh, density)
    F_global += apply_pore_pressure_3d(mesh, water_table_depth, pore_pressure)
    F_global += apply_surcharge_load_3d(mesh, shapefile_path, surcharge_load)
    F_global += apply_seismic_load_3d(mesh, seismic_coefficient)
    print("All loads applied.")
    return F_global


In [11]:
F_global = apply_loads_3d(mesh, density, water_table_depth, pore_pressure, surcharge_load, seismic_coefficient)

Applying all loads...
Applying gravity load...
Gravity load applied.
Applying predefined pore water pressure normal to each cell face...
Predefined pore water pressure applied normal to each cell face.
Reading shapefile...
Applying surcharge load to areas defined by the polygon...
Point [9.53651736e+05 2.42998109e+06 5.90091003e+02] is within polygon.
Point [9.53653155e+05 2.42998109e+06 5.89896851e+02] is within polygon.
Point [9.53654573e+05 2.42998109e+06 5.89579102e+02] is within polygon.
Point [9.53655991e+05 2.42998109e+06 5.89504089e+02] is within polygon.
Point [9.53651736e+05 2.42997967e+06 5.90046509e+02] is within polygon.
Point [9.53653155e+05 2.42997967e+06 5.89848328e+02] is within polygon.
Point [9.53654573e+05 2.42997967e+06 5.89703674e+02] is within polygon.
Point [9.53655991e+05 2.42997967e+06 5.89697327e+02] is within polygon.
Point [9.53657410e+05 2.42997967e+06 5.89603760e+02] is within polygon.
Point [9.53651736e+05 2.42997825e+06 5.90067749e+02] is within polygon

In [12]:


def plot_load(mesh, force_vector, title, color='blue', magnitude=0.007):
    """
    Plot the applied load on the mesh using PyVista.
    
    Parameters:
    mesh (pyvista.PolyData): The mesh to visualize.
    force_vector (cp.ndarray): The force vector to visualize (flattened array).
    title (str): The title of the plot.
    color (str): The color of the arrows representing the forces.
    magnitude (float): The scaling factor for the arrow size.
    """
    # Convert force vector from CuPy to NumPy for plotting
    force_vector_np = cp.asnumpy(force_vector.reshape(-1, 3))
    
    # Initialize PyVista plotter
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, color='white', opacity=0.5)
    plotter.add_arrows(mesh.points, force_vector_np, mag=magnitude, color=color, label=title)
    
    # Show the plot with title and specified window size
    plotter.show(title=title, window_size=[800, 600])

def plot_all_loads(mesh, F_gravity, F_pore_pressure, F_surcharge, F_seismic):
    """
    Plot all the applied loads separately on the mesh.
    
    Parameters:
    mesh (pyvista.PolyData): The mesh to visualize.
    F_gravity (cp.ndarray): Gravity load vector.
    F_pore_pressure (cp.ndarray): Pore pressure load vector.
    F_surcharge (cp.ndarray): Surcharge load vector.
    F_seismic (cp.ndarray): Seismic load vector.
    """
    # Plot gravity load
    plot_load(mesh, F_gravity, "Gravity Load", color='blue')
    
    # Plot pore pressure load
    plot_load(mesh, F_pore_pressure, "Pore Pressure Load", color='green')
    
    # Plot surcharge load
    plot_load(mesh, F_surcharge, "Surcharge Load", color='red')
    
    # Plot seismic load
    plot_load(mesh, F_seismic, "Seismic Load", color='purple')



F_gravity = apply_gravity_load_3d(mesh, density)
F_pore_pressure = apply_pore_pressure_3d(mesh, water_table_depth, pore_pressure)
F_surcharge = apply_surcharge_load_3d(mesh, shapefile_path, surcharge_load)
F_seismic = apply_seismic_load_3d(mesh, seismic_coefficient)

 #Plot all loads
plot_all_loads(mesh, F_gravity, F_pore_pressure, F_surcharge, F_seismic)


Applying gravity load...
Gravity load applied.
Applying predefined pore water pressure normal to each cell face...
Predefined pore water pressure applied normal to each cell face.
Reading shapefile...
Applying surcharge load to areas defined by the polygon...
Point [9.53651736e+05 2.42998109e+06 5.90091003e+02] is within polygon.
Point [9.53653155e+05 2.42998109e+06 5.89896851e+02] is within polygon.
Point [9.53654573e+05 2.42998109e+06 5.89579102e+02] is within polygon.
Point [9.53655991e+05 2.42998109e+06 5.89504089e+02] is within polygon.
Point [9.53651736e+05 2.42997967e+06 5.90046509e+02] is within polygon.
Point [9.53653155e+05 2.42997967e+06 5.89848328e+02] is within polygon.
Point [9.53654573e+05 2.42997967e+06 5.89703674e+02] is within polygon.
Point [9.53655991e+05 2.42997967e+06 5.89697327e+02] is within polygon.
Point [9.53657410e+05 2.42997967e+06 5.89603760e+02] is within polygon.
Point [9.53651736e+05 2.42997825e+06 5.90067749e+02] is within polygon.
Point [9.53653155e+0

Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x2539f3330d0_0&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x2551f6b68d0_1&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x2551f313510_2&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x2552613b650_3&reconnect=auto" class="pyvis…

In [13]:


def calculate_resultant_forces(mesh, F_global):
    """
    Calculate the resultant force for each cell.
    
    Parameters:
    mesh (pyvista.PolyData): The mesh to which the load is applied.
    F_global (cp.ndarray): The global force vector (flattened array).
    
    Returns:
    tuple: (resultant_forces, magnitudes)
        resultant_forces (np.ndarray): The resultant force vectors for each cell.
        magnitudes (np.ndarray): The magnitude of the resultant forces.
    """
    F_global_np = F_global.get().reshape(-1, 3)  # Convert and reshape to (n_points, 3)

    # Initialize arrays to store resultant forces and magnitudes
    resultant_forces = []
    magnitudes = []

    # Iterate through each cell to calculate the resultant force
    for i in range(mesh.n_cells):
        cell = mesh.get_cell(i)
        cell_point_ids = cell.point_ids
        force_vector = F_global_np[cell_point_ids].sum(axis=0)  # Sum forces at all nodes of the cell
        resultant_forces.append(force_vector)
        magnitudes.append(np.linalg.norm(force_vector))  # Calculate the magnitude of the force vector

    return np.array(resultant_forces), np.array(magnitudes)

def plot_resultant_forces(mesh, resultant_forces, magnitudes):
    """
    Plot the resultant forces with color gradient based on the magnitude.
    
    Parameters:
    mesh (pyvista.PolyData): The mesh to which the load is applied.
    resultant_forces (np.ndarray): The resultant force vectors for each cell.
    magnitudes (np.ndarray): The magnitude of the resultant forces.
    """
    # Normalize magnitudes for color mapping
    cmap = plt.colormaps.get_cmap('viridis')
    normalized_magnitudes = (magnitudes - magnitudes.min()) / (magnitudes.max() - magnitudes.min())
    colors = cmap(normalized_magnitudes)[:, :3]  # Get RGB colors from the colormap

    # Create a point cloud of the cell centers
    cell_centers = mesh.cell_centers().points

    # Create a PyVista vector field for the resultant forces
    arrows = pv.PolyData(cell_centers)
    arrows["vectors"] = resultant_forces
    arrows["magnitudes"] = magnitudes

    # Create glyphs (arrows) with the vectors and assign colors based on magnitudes
    glyphs = arrows.glyph(orient="vectors", scale="magnitudes", factor=0.00009)

    # Repeat the colors array to match the number of points in glyphs
    repeated_colors = np.repeat(colors, glyphs.n_points // len(colors), axis=0)
    glyphs.point_data["colors"] = repeated_colors

    # Initialize the plotter
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', opacity=0.6)  # Plot the original mesh
    plotter.add_mesh(glyphs, scalars="colors", colormap=None)  # Apply colors directly without a colormap
    
    # Add a title and show the plot
    plotter.add_title('Resultant Force Vectors with Color Gradient')
    plotter.show()


resultant_forces, magnitudes = calculate_resultant_forces(mesh, F_global)
plot_resultant_forces(mesh, resultant_forces, magnitudes)


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x255261bd0d0_4&reconnect=auto" class="pyvis…



'''def identify_fixed_nodes_below_elevation(mesh, elevation_threshold):
    fixed_nodes = []
    below_threshold_nodes = []
    x_boundary_nodes = []
    y_boundary_nodes = []
    
    # Find minimum and maximum coordinates
    min_x = mesh.points[:, 0].min()
    max_x = mesh.points[:, 0].max()
    min_y = mesh.points[:, 1].min()
    max_y = mesh.points[:, 1].max()
    
    # Loop through each node and check if it's a fixed node
    for i, node in enumerate(mesh.points):
        # Fix nodes below the specified elevation
        if node[2] <= elevation_threshold:
            fixed_nodes.append(i)
            below_threshold_nodes.append(i)
        # Fix nodes on vertical boundaries in x direction
        elif node[0] == min_x or node[0] == max_x:
            fixed_nodes.append(i)
            x_boundary_nodes.append(i)
        # Fix nodes on vertical boundaries in y direction
        elif node[1] == min_y or node[1] == max_y:
            fixed_nodes.append(i)
            y_boundary_nodes.append(i)
    
    print(f"Total number of nodes: {mesh.points.shape[0]}")
    print(f"Number of nodes below elevation {elevation_threshold}: {len(below_threshold_nodes)}")
    print(f"Number of nodes with support on vertical boundaries in x direction: {len(x_boundary_nodes)}")
    print(f"Number of nodes with support on vertical boundaries in y direction: {len(y_boundary_nodes)}")
    
    return fixed_nodes, below_threshold_nodes, x_boundary_nodes, y_boundary_nodes

def apply_boundary_conditions_surface(K_global, F_global, fixed_nodes):
    print("Applying boundary conditions...")

    num_dofs_per_node = 3
    fixed_dofs = cp.array([node * num_dofs_per_node + i for node in fixed_nodes for i in range(num_dofs_per_node)])
    
    # Create a mask for non-fixed DOFs
    non_fixed_dofs = cp.ones(K_global.shape[0], dtype=bool)
    non_fixed_dofs[fixed_dofs] = False

    # Zero out the rows and columns for fixed DOFs in K_global
    K_global[fixed_dofs, :] = 0
    K_global[:, fixed_dofs] = 0

    # Set diagonal for fixed DOFs
    K_global[fixed_dofs, fixed_dofs] = 1

    # Zero out the corresponding entries in F_global
    F_global[fixed_dofs] = 0

    print(f"K_global shape: {K_global.shape}, F_global shape: {F_global.shape}")
    print("Boundary conditions applied.")
    return K_global, F_global'''






In [14]:


def identify_fixed_nodes_below_elevation(mesh, elevation_threshold):
    fixed_nodes = []
    below_threshold_nodes = []
    boundary_nodes = []
    
    # Find minimum and maximum coordinates
    min_x = mesh.points[:, 0].min()
    max_x = mesh.points[:, 0].max()
    min_y = mesh.points[:, 1].min()
    max_y = mesh.points[:, 1].max()
    
    # Loop through each node and check if it's a fixed node
    for i, node in enumerate(mesh.points):
        # Fix nodes below the specified elevation
        if node[2] <= elevation_threshold:
            fixed_nodes.append(i)
            below_threshold_nodes.append(i)
        # Fix nodes on all boundaries (x and y directions)
        if node[0] == min_x or node[0] == max_x or node[1] == min_y or node[1] == max_y:
            fixed_nodes.append(i)
            boundary_nodes.append(i)
    
    # Print counts
    print(f"Total number of nodes: {mesh.points.shape[0]}")
    print(f"Number of nodes below elevation {elevation_threshold}: {len(below_threshold_nodes)}")
    print(f"Number of nodes on boundaries: {len(boundary_nodes)}")
    print(f"Total number of fixed nodes: {len(fixed_nodes)}")
    
    return fixed_nodes, below_threshold_nodes, boundary_nodes

def apply_boundary_conditions_surface(K_global, F_global, fixed_nodes):
    print("Applying boundary conditions...")

    num_dofs_per_node = 3
    fixed_dofs = cp.array([node * num_dofs_per_node + i for node in fixed_nodes for i in range(num_dofs_per_node)])
    
    # Create a mask for non-fixed DOFs
    non_fixed_dofs = cp.ones(K_global.shape[0], dtype=bool)
    non_fixed_dofs[fixed_dofs] = False

    # Zero out the rows and columns for fixed DOFs in K_global
    K_global[fixed_dofs, :] = 0
    K_global[:, fixed_dofs] = 0

    # Set diagonal for fixed DOFs
    K_global[fixed_dofs, fixed_dofs] = 1

    # Zero out the corresponding entries in F_global
    F_global[fixed_dofs] = 0

    print(f"K_global shape: {K_global.shape}, F_global shape: {F_global.shape}")
    print("Boundary conditions applied.")
    return K_global, F_global




In [15]:


# Define the elevation threshold for fixing nodes
elevation_threshold = 590  # Adjust this value as needed

# Identify fixed nodes based on the elevation threshold and boundary conditions
fixed_nodes, below_threshold_nodes, boundary_nodes = identify_fixed_nodes_below_elevation(mesh, elevation_threshold)

# Apply boundary conditions (if needed)
K_global, F_global = apply_boundary_conditions_surface(K_global, F_global, fixed_nodes)

# Plot the mesh and highlight fixed nodes
plotter = pv.Plotter()
plotter.add_mesh(mesh, color='white', opacity=0.5)

# Highlight fixed nodes
plotter.add_points(mesh.points[below_threshold_nodes], color='blue', point_size=10, render_points_as_spheres=True, label='Nodes Below Threshold')
plotter.add_points(mesh.points[boundary_nodes], color='red', point_size=10, render_points_as_spheres=True, label='Boundary Nodes')

# Add a legend
plotter.add_legend()

# Show the plot
plotter.show()

Total number of nodes: 3055
Number of nodes below elevation 590: 96
Number of nodes on boundaries: 220
Total number of fixed nodes: 316
Applying boundary conditions...


C:\Users\Sekhar\.conda\envs\PDAL\Lib\site-packages\cupyx\scipy\sparse\_compressed.py:548: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '


K_global shape: (9165, 9165), F_global shape: (9165,)
Boundary conditions applied.


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x2539f39d250_5&reconnect=auto" class="pyvis…

In [16]:
# Solve Displacements for 3D
def solve_displacements_3d(K_global, F_global, method='gmres'):
    print(f"Solving for displacements using {method.upper()} on GPU...")

    # Ensure the global stiffness matrix is in CSR format for efficient matrix-vector operations
    K_global = K_global.tocsr()

    def matvec(x):
        return K_global.dot(x)

    K_operator = LinearOperator(K_global.shape, matvec)

    print(f"K_global size: {K_global.shape}, F_global size: {F_global.shape}")
    if method == 'gmres':
        U_global, info = gmres(K_operator, F_global, tol=1e-8, maxiter=3135)
    elif method == 'bicgstab':
        print("Falling back to CPU-based BiCGSTAB solver...")
        K_global_cpu = K_global.get()  # Transfer to CPU
        F_global_cpu = F_global.get()  # Transfer to CPU
        U_global_cpu, info = bicgstab(K_global_cpu, F_global_cpu, tol=1e-8, maxiter=3135)
        U_global = cp.asarray(U_global_cpu)  # Transfer back to GPU
    else:
        raise ValueError(f"Unknown method: {method}")

    if info != 0:
        print(f"{method.upper()} solver did not converge. Info: {info}")
    else:
        print(f"Displacements solved using {method.upper()}.")
    
    return U_global

In [17]:
U_global = solve_displacements_3d(K_global, F_global)

Solving for displacements using GMRES on GPU...
K_global size: (9165, 9165), F_global size: (9165,)
Displacements solved using GMRES.


In [33]:

# Convert U_global to a NumPy array
U_global_np = cp.asnumpy(U_global).reshape((-1, 3))

# Get the original nodal positions
original_points = mesh.points

# Compute the deformed nodal positions
deformed_points = original_points + U_global_np

# Create a new mesh with the deformed points
deformed_mesh = mesh.copy()
deformed_mesh.points = deformed_points

# Create a plotter object
plotter = pv.Plotter()

# Add the original mesh
plotter.add_mesh(mesh, color='white', opacity=1, label='Original Mesh')

# Add the deformed mesh
plotter.add_mesh(deformed_mesh, color='red',opacity=0.5, label='Deformed Mesh')

# Add a legend
plotter.add_legend()

# Show the plot
plotter.show()


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x255b4fca890_16&reconnect=auto" class="pyvi…

In [19]:
def compute_stress_tensor_3d(mesh, U_global, E, nu):
    """
    Compute the stress tensor for each element in a 3D surface mesh.
    
    Parameters:
    - mesh: PyVista mesh object representing the 3D surface mesh.
    - U_global: Global displacement vector (CuPy array).
    - E: Young's modulus (float).
    - nu: Poisson's ratio (float).

    Returns:
    - stresses: CuPy array containing stress components for each element.
    """
    print("Computing stress tensor for 3D surface mesh...")
    stresses = cp.zeros((mesh.n_cells, 3))  # Store 3 components of the stress tensor for each cell
    dN_dxi = cp.array([
        [-1, -1],
        [1, 0],
        [0, 1]
    ])
    
    mesh_points = cp.array(mesh.points)

    for i in tqdm(range(mesh.n_cells)):
        cell = mesh.get_cell(i)
        nodes = cp.array(cell.point_ids).astype(int)  # Use CuPy array for indexing
        element_nodes = mesh_points[nodes]
        
        J = compute_jacobian_3d(element_nodes, dN_dxi)
        det_J = cp.linalg.det(cp.dot(J, J.T))
        
        if cp.abs(det_J) < 1e-12:
            print(f"Warning: Degenerate element detected at cell {i}. Skipping.")
            continue
        
        J_inv = cp.linalg.inv(J[:2, :2])  # Use only 2x2 Jacobian for in-plane deformation
        B = compute_B_matrix_3d(J_inv, dN_dxi)
        C = compute_C_matrix_3d(E, nu)
        
        U_elem = U_global[nodes.repeat(3) * 3 + cp.tile(cp.arange(3), len(nodes))]
        epsilon = cp.dot(B, U_elem)
        sigma = cp.dot(C, epsilon)
        stresses[i, :] = sigma
    
    print("Stress tensor computed for 3D surface mesh.")
    return stresses


In [20]:
stresses = compute_stress_tensor_3d(mesh, U_global, E, nu)

Computing stress tensor for 3D surface mesh...


100%|█████████████████████████████████████████████████████████████████████████████| 5888/5888 [00:13<00:00, 437.57it/s]


Stress tensor computed for 3D surface mesh.


In [21]:
def calculate_normal_stress_3d(stress_tensor, plane_normal):
    """Calculate the normal stress on a plane given by plane_normal."""
    nx, ny, nz = plane_normal
    sigma_xx, sigma_yy, sigma_zz = stress_tensor
    normal_stress = (nx**2) * sigma_xx + (ny**2) * sigma_yy + (nz**2) * sigma_zz
    return normal_stress

def calculate_shear_strength(cohesion, normal_stress, friction_angle):
    """Calculate the shear strength using the Mohr-Coulomb failure criterion."""
    return cohesion + normal_stress * cp.tan(cp.radians(friction_angle))

def compute_fos_3d(stresses, cohesions, friction_angles, mesh, plane_normal=(0, 0, 1)):
    """Calculate the Factor of Safety (FoS) for each element in the mesh."""
    print("Calculating Factor of Safety (FoS) for 3D surface mesh...")
    fos = cp.zeros(mesh.n_cells)
    
    for i in tqdm(range(mesh.n_cells)):
        stress_tensor = stresses[i]
        normal_stress = calculate_normal_stress_3d(stress_tensor, plane_normal)
        cohesion = cohesions[i]
        friction_angle = friction_angles[i]
        shear_strength = calculate_shear_strength(cohesion, normal_stress, friction_angle)
        
        if normal_stress > 0:
            fos[i] = shear_strength / normal_stress
        else:
            fos[i] = float('inf')  # No failure if stress is zero or negative
    
    print("Factor of Safety (FoS) calculated for 3D surface mesh.")
    return fos



In [22]:
cohesion_array = cp.array([cohesion] * mesh.n_cells)  # Cohesion values for each element
friction_angle_array = cp.array([friction_angle] * mesh.n_cells)  # Friction angles for each element

fos = compute_fos_3d(stresses, cohesion_array, friction_angle_array, mesh)

Calculating Factor of Safety (FoS) for 3D surface mesh...


100%|████████████████████████████████████████████████████████████████████████████| 5888/5888 [00:01<00:00, 4225.91it/s]

Factor of Safety (FoS) calculated for 3D surface mesh.


In [34]:
def plot_fos_2d(mesh, fos, vmin=None, vmax=None):
    # Convert CuPy array back to NumPy for plotting
    fos_numpy = cp.asnumpy(fos)
    mesh.cell_data['FoS'] = fos_numpy  
    
    # Determine min and max FoS values
    min_fos = fos_numpy.min()
    max_fos = fos_numpy.max()
    
    # Print min and max FoS values
    print(f"Minimum FoS value: {min_fos}")
    print(f"Maximum FoS value: {max_fos}")
    
    # Define color limits
    if vmin is None:
        vmin = min_fos
    if vmax is None:
        vmax = max_fos

    plotter = pv.Plotter()
    plotter.add_mesh(mesh, scalars='FoS', cmap='coolwarm', show_edges=True, clim=[vmin, vmax])
    plotter.add_axes()
    plotter.add_title("Factor of Safety (FoS) for 2D Mesh")
    plotter.show()

# Example usage
plot_fos_2d(mesh, fos, vmin=0, vmax=1)  # Set vmin and vmax as per your requirements



Minimum FoS value: 0.8472863882156269
Maximum FoS value: inf


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x255c4451650_17&reconnect=auto" class="pyvi…

In [24]:
def identify_failure_cells_and_normals(mesh, fos):
    """
    Identify failure cells and their respective normal vectors.

    Parameters:
    - mesh: PyVista mesh object
    - fos: CuPy array of Factor of Safety values

    Returns:
    - failure_cells: List of indices of failure cells
    - vector_normals: List of normal vectors of failure cells
    """
    print("Identifying failure cells...")
    failure_cells = []
    vector_normals = []

    for i in range(mesh.n_cells):
        if fos[i] < 1:  # Assuming FoS < 1 indicates failure
            failure_cells.append(i)
            normal = mesh.cell_normals[i]
            vector_normals.append(normal)

    print(f"Identified {len(failure_cells)} failure cells.")
    return failure_cells, vector_normals

# Usage
failure_cells, vector_normals = identify_failure_cells_and_normals(mesh, fos)

# Plotting the failure cells
def plot_failure_cells(mesh, failure_cells):
    plotter = pv.Plotter()
    failure_mesh = mesh.extract_cells(failure_cells)
    plotter.add_mesh(failure_mesh, color='red', show_edges=True)
    plotter.add_axes()
    plotter.add_title("Failure Cells in 2D Mesh")
    plotter.show()

plot_failure_cells(mesh, failure_cells)


Identifying failure cells...
Identified 72 failure cells.


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x2559dadc0d0_8&reconnect=auto" class="pyvis…

In [25]:


# Ensure we have the failure cells identified already
# We assume failure_cells and fos have been calculated from previous cells

# Function to extract connected components of failure surfaces
def extract_connected_failure_surfaces(mesh, failure_cells):
    """
    Extract connected components of failure surfaces from a mesh.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - failure_cells: List of indices of failure cells.

    Returns:
    - components_meshes: List of PyVista mesh objects, each representing a connected failure surface.
    """
    print("Extracting connected components of failure surfaces...")

    # Extract failure cells into a new mesh
    failure_mesh = mesh.extract_cells(failure_cells)
    
    # Extract connected components
    connected_components = failure_mesh.connectivity()

    # Create a list to hold individual component meshes
    components_meshes = []
    for label in np.unique(connected_components.cell_data['RegionId']):
        component_mesh = failure_mesh.extract_cells(np.where(connected_components.cell_data['RegionId'] == label)[0])
        components_meshes.append(component_mesh)
    
    print(f"Found {len(components_meshes)} connected components of failure surfaces.")
    return components_meshes

# Extract connected failure surfaces
components_meshes = extract_connected_failure_surfaces(mesh, failure_cells)

# Function to visualize the entire mesh with failure and non-failure cells
def visualize_mesh_with_fos(mesh, fos, failure_cells):
    """
    Visualize the entire mesh with a color gradient based on FoS for failure cells.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - fos: Array of FoS values for each cell in the mesh.
    - failure_cells: List of indices of failure cells.
    """
    plotter = pv.Plotter()

    # Convert fos to numpy array if it is a CuPy array
    if isinstance(fos, cp.ndarray):
        fos = fos.get()

    # Create an array to store cell colors
    cell_colors = np.full(mesh.n_cells, np.nan)  # Use NaN for non-failure cells

    # Assign FoS values to failure cells
    cell_colors[failure_cells] = fos[failure_cells]

    # Add the mesh to the plotter with the cell colors
    mesh.cell_data['FoS'] = cell_colors
    plotter.add_mesh(mesh, scalars='FoS', cmap='spectral', show_edges=True, nan_opacity=0.7)

    plotter.add_axes()
    plotter.add_title("Mesh with Failure Cells Colored by FoS")
    plotter.show()

# Visualize the mesh with FoS-based color gradient for failure cells
visualize_mesh_with_fos(mesh, fos, failure_cells)


Extracting connected components of failure surfaces...
Found 6 connected components of failure surfaces.


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x2559da3b7d0_9&reconnect=auto" class="pyvis…

In [26]:


# Function to calculate displacement magnitude and direction for failure cells
def calculate_displacement_magnitude_direction(mesh, U_global, failure_cells):
    """
    Calculate displacement magnitude and direction for each failure cell.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - U_global: CuPy array of global displacement vector.
    - failure_cells: List of indices of failure cells.

    Returns:
    - displacement_magnitudes: NumPy array of displacement magnitudes for each failure cell.
    - displacement_vectors: NumPy array of displacement vectors (3D) for each failure cell.
    """
    print("Calculating displacement magnitude and direction for failure cells...")

    # Ensure displacement is on GPU with CuPy
    if isinstance(U_global, np.ndarray):
        U_global = cp.asarray(U_global)
    
    # Initialize arrays to store displacement magnitudes and vectors
    displacement_magnitudes = []
    displacement_vectors = []

    # Loop over each failure cell
    for cell_idx in failure_cells:
        cell = mesh.get_cell(cell_idx)
        nodes = cp.array(cell.point_ids).astype(int)  # Convert nodes to CuPy array
        
        # Extract displacement vectors for the nodes of the cell
        U_cell = U_global[nodes.repeat(2) * 2 + cp.tile(cp.arange(2), len(nodes))]
        
        # Calculate mean displacement vector for the cell
        mean_disp = cp.mean(U_cell.reshape(-1, 2), axis=0)
        magnitude = cp.linalg.norm(mean_disp)
        
        # Append the 3D vector (assuming 2D mesh, z-component is 0)
        displacement_vectors.append(np.array([mean_disp[0].get(), mean_disp[1].get(), 0.0]))  # Convert to 3D vector
        displacement_magnitudes.append(magnitude.get())  # Convert to NumPy for plotting
    
    print("Displacement calculation complete.")
    return np.array(displacement_magnitudes), np.array(displacement_vectors)

# Function to compute resultant displacement for each compound
def compute_resultant_displacement_for_compounds(mesh, components_meshes, displacement_vectors):
    """
    Compute the resultant displacement magnitude and direction for each connected component of failure surfaces.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - components_meshes: List of PyVista mesh objects, each representing a connected failure surface.
    - displacement_vectors: NumPy array of displacement vectors for each failure cell.

    Returns:
    - resultant_magnitudes: List of resultant displacement magnitudes for each component.
    - resultant_directions: List of resultant displacement directions (unit vectors) for each component.
    - centroids: List of centroids for each component.
    """
    print("Computing resultant displacement for each compound...")
    
    resultant_magnitudes = []
    resultant_directions = []
    centroids = []

    for component_mesh in components_meshes:
        # Get indices of cells in the component
        component_cell_ids = component_mesh.cell_data['vtkOriginalCellIds']

        # Sum up displacement vectors for the entire component
        resultant_vector = np.sum(displacement_vectors[component_cell_ids], axis=0)
        
        # Compute magnitude of the resultant vector
        resultant_magnitude = np.linalg.norm(resultant_vector)

        # Compute unit vector for the direction (if magnitude is not zero)
        if resultant_magnitude != 0:
            resultant_direction = resultant_vector / resultant_magnitude
        else:
            resultant_direction = np.array([0.0, 0.0, 0.0])
        
        resultant_magnitudes.append(resultant_magnitude)
        resultant_directions.append(resultant_direction)
        
        # Compute centroid of the component for arrow placement
        centroid = component_mesh.cell_centers().points.mean(axis=0)
        centroids.append(centroid)
    
    print("Resultant displacement calculation for each compound is complete.")
    return resultant_magnitudes, resultant_directions, centroids

# Function to visualize compounds with resultant displacements
def visualize_compounds_with_displacement(mesh, components_meshes, resultant_magnitudes, resultant_directions, centroids):
    """
    Visualize the compounds with resultant displacement vectors and magnitudes.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - components_meshes: List of PyVista mesh objects, each representing a connected failure surface.
    - resultant_magnitudes: List of resultant displacement magnitudes for each component.
    - resultant_directions: List of resultant displacement directions (unit vectors) for each component.
    - centroids: List of centroids for each component.
    """
    print("Visualizing compounds with resultant displacement vectors and magnitudes...")

    # Initialize the plotter
    plotter = pv.Plotter()

    # Add the original mesh to the plotter
    plotter.add_mesh(mesh, color='white', show_edges=True, opacity=0.8)

    # Normalize magnitudes for arrow scaling
    max_magnitude = max(resultant_magnitudes) if resultant_magnitudes else 1
    scale_factor = 100/ max_magnitude  # Adjust the scaling factor as needed

    # Loop through each component to add to the plotter
    for i, component_mesh in enumerate(components_meshes):
        # Add the component mesh with a color gradient based on resultant magnitudes
        component_mesh['Resultant Magnitude'] = np.full(component_mesh.n_cells, resultant_magnitudes[i])
        plotter.add_mesh(component_mesh, scalars='Resultant Magnitude', cmap='spectral', show_edges=True, opacity=1)

        # Add the resultant displacement vector as an arrow, scaled by magnitude
        arrow_scale = resultant_magnitudes[i] * scale_factor
        plotter.add_arrows(centroids[i].reshape(1, 3), resultant_directions[i].reshape(1, 3), mag=arrow_scale, color='green')
    
    plotter.add_axes()
    plotter.add_title("Compounds with Resultant Displacement Vectors and Magnitudes")
    plotter.show()

# Usage
displacement_magnitudes, displacement_vectors = calculate_displacement_magnitude_direction(mesh, U_global, failure_cells)
components_meshes = extract_connected_failure_surfaces(mesh, failure_cells)  # This function should already be defined in your code

# Calculate resultant displacement for each compound
resultant_magnitudes, resultant_directions, centroids = compute_resultant_displacement_for_compounds(mesh, components_meshes, displacement_vectors)

# Visualize the compounds with displacement vectors and magnitudes
visualize_compounds_with_displacement(mesh, components_meshes, resultant_magnitudes, resultant_directions, centroids)


Calculating displacement magnitude and direction for failure cells...
Displacement calculation complete.
Extracting connected components of failure surfaces...
Found 6 connected components of failure surfaces.
Computing resultant displacement for each compound...
Resultant displacement calculation for each compound is complete.
Visualizing compounds with resultant displacement vectors and magnitudes...


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x255a193b510_10&reconnect=auto" class="pyvi…

In [27]:


# Function to calculate displacement magnitude and direction for failure cells
def calculate_displacement_magnitude_direction(mesh, U_global, failure_cells):
    """
    Calculate displacement magnitude and direction for each failure cell.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - U_global: CuPy array of global displacement vector.
    - failure_cells: List of indices of failure cells.

    Returns:
    - displacement_magnitudes: NumPy array of displacement magnitudes for each failure cell.
    - displacement_vectors: NumPy array of displacement vectors (3D) for each failure cell.
    """
    print("Calculating displacement magnitude and direction for failure cells...")

    # Ensure displacement is on GPU with CuPy
    if isinstance(U_global, np.ndarray):
        U_global = cp.asarray(U_global)
    
    # Initialize arrays to store displacement magnitudes and vectors
    displacement_magnitudes = []
    displacement_vectors = []

    # Loop over each failure cell
    for cell_idx in failure_cells:
        cell = mesh.get_cell(cell_idx)
        nodes = cp.array(cell.point_ids).astype(int)  # Convert nodes to CuPy array
        
        # Extract displacement vectors for the nodes of the cell
        U_cell = U_global[nodes.repeat(2) * 2 + cp.tile(cp.arange(2), len(nodes))]
        
        # Calculate mean displacement vector for the cell
        mean_disp = cp.mean(U_cell.reshape(-1, 2), axis=0)
        magnitude = cp.linalg.norm(mean_disp)
        
        # Append the 3D vector (assuming 2D mesh, z-component is 0)
        displacement_vectors.append(np.array([mean_disp[0].get(), mean_disp[1].get(), 0.0]))  # Convert to 3D vector
        displacement_magnitudes.append(magnitude.get())  # Convert to NumPy for plotting
    
    print("Displacement calculation complete.")
    return np.array(displacement_magnitudes), np.array(displacement_vectors)

# Function to visualize mesh with displacement vectors and magnitudes
def visualize_displacement_vectors(mesh, failure_cells, displacement_magnitudes, displacement_vectors):
    """
    Visualize the mesh with displacement vectors as arrows and magnitudes as a color gradient.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - failure_cells: List of indices of failure cells.
    - displacement_magnitudes: NumPy array of displacement magnitudes for each failure cell.
    - displacement_vectors: NumPy array of displacement vectors for each failure cell.
    """
    print("Visualizing displacement vectors and magnitudes...")

    # Initialize the plotter
    plotter = pv.Plotter()
    
    # Create an array to store cell magnitudes for coloring
    cell_colors = np.full(mesh.n_cells, np.nan)  # Use NaN for non-failure cells
    
    # Assign displacement magnitudes to failure cells
    cell_colors[failure_cells] = displacement_magnitudes
    
    # Add the mesh with magnitudes to the plotter
    mesh.cell_data['Displacement Magnitude'] = cell_colors
    plotter.add_mesh(mesh, scalars='Displacement Magnitude', cmap='plasma', show_edges=True, nan_opacity=0.5)
    
    # Extract the failure mesh for plotting vectors
    failure_mesh = mesh.extract_cells(failure_cells)
    centroids = failure_mesh.cell_centers().points

    # Ensure that the number of centroids matches the number of displacement vectors
    if len(centroids) == len(displacement_vectors):
        # Add displacement vectors as arrows
        plotter.add_arrows(centroids, displacement_vectors, mag=10, color='black')  # Adjust 'mag' for scaling arrows
    else:
        print("Error: The number of centroids does not match the number of displacement vectors. Check your data.")
    
    plotter.add_axes()
    plotter.add_title("Mesh with Displacement Vectors and Magnitudes")
    plotter.show()

# Usage
displacement_magnitudes, displacement_vectors = calculate_displacement_magnitude_direction(mesh, U_global, failure_cells)
visualize_displacement_vectors(mesh, failure_cells, displacement_magnitudes, displacement_vectors)


Calculating displacement magnitude and direction for failure cells...
Displacement calculation complete.
Visualizing displacement vectors and magnitudes...


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x2553e315fd0_11&reconnect=auto" class="pyvi…

In [28]:


def classify_failure_type(component_mesh, displacement_vectors, centroid):
    """
    Classify the failure type of a compound based on geometry and displacement patterns.

    Parameters:
    - component_mesh: PyVista mesh object representing a connected failure surface.
    - displacement_vectors: NumPy array of displacement vectors for each cell in the compound.
    - centroid: Centroid of the connected component for reference.

    Returns:
    - failure_type: String indicating the type of failure.
    """
    # Extract the surface of the component mesh
    surface_mesh = component_mesh.extract_surface()
    
    # Calculate cell normals explicitly for the surface mesh
    surface_mesh_with_normals = surface_mesh.compute_normals(cell_normals=True)
    cell_normals = surface_mesh_with_normals['Normals']

    unique_normals = np.unique(cell_normals, axis=0)
    num_unique_normals = len(unique_normals)
    
    # Evaluate displacement patterns
    avg_displacement_vector = np.mean(displacement_vectors, axis=0)
    displacement_magnitude = np.linalg.norm(avg_displacement_vector)
    
    # Calculate angle of displacement direction with vertical axis (z-axis)
    vertical_axis = np.array([0, 0, 1])
    angle_with_vertical = np.arccos(np.clip(np.dot(avg_displacement_vector, vertical_axis) / displacement_magnitude, -1.0, 1.0))
    angle_with_vertical_deg = np.degrees(angle_with_vertical)
    
    # Criteria for classification
    if num_unique_normals == 1:
        if angle_with_vertical_deg > 75:
            return 'Toppling Failure'
        else:
            return 'Planar Failure'
    elif num_unique_normals == 2:
        if is_multiple_wedge_failure(surface_mesh, unique_normals):
            return 'Multiple Wedge Failure'
        return 'Wedge Failure'
    elif num_unique_normals > 2:
        if is_step_path_failure(component_mesh):
            return 'Step-Path Failure'
        elif is_circular_failure(component_mesh, centroid):
            return 'Circular Failure'
        elif is_rotational_failure(displacement_vectors):
            return 'Rotational Failure'
        else:
            return 'Irregular Failure'
    else:
        return 'Compound Failure'

def is_multiple_wedge_failure(surface_mesh, unique_normals):
    """
    Check if the failure is a multiple wedge failure.

    Parameters:
    - surface_mesh: Surface representation of the component mesh.
    - unique_normals: Array of unique normals from the component mesh.

    Returns:
    - is_multiple_wedge: Boolean indicating if the failure is a multiple wedge failure.
    """
    # A multiple wedge failure would involve more than two intersecting planes
    # Check if there are three or more planes that intersect at the surface
    return len(unique_normals) >= 3

def is_step_path_failure(component_mesh):
    """
    Determine if the failure is a step-path failure.

    Parameters:
    - component_mesh: PyVista mesh object representing a connected failure surface.

    Returns:
    - is_step_path: Boolean indicating if the failure is a step-path failure.
    """
    # Check for zigzag pattern in the geometry of the failure surface
    # Typically involves alternating planes or layers
    # This could be identified by analyzing the pattern of normals or connectivity of the mesh
    # Placeholder logic - needs domain-specific checks
    return False  # Implement domain-specific check

def is_rotational_failure(displacement_vectors):
    """
    Determine if the failure surface is undergoing rotational failure.

    Parameters:
    - displacement_vectors: NumPy array of displacement vectors for each cell in the compound.

    Returns:
    - is_rotational: Boolean indicating if the failure is rotational.
    """
    # Check if displacement vectors form a rotational pattern around a pivot
    # Typically involves circular displacement with varying angles
    # Placeholder logic - needs domain-specific checks
    rotation_center = np.mean(displacement_vectors, axis=0)
    rotations = displacement_vectors - rotation_center
    variances = np.var(rotations, axis=0)
    is_rotational = np.all(variances < 0.1)  # Example threshold; refine based on domain
    return is_rotational

def is_circular_failure(component_mesh, centroid):
    """
    Determine if the failure surface is circular or arc-shaped.

    Parameters:
    - component_mesh: PyVista mesh object representing a connected failure surface.
    - centroid: Centroid of the connected component for reference.

    Returns:
    - is_circular: Boolean indicating if the failure is circular.
    """
    # Compute distances from centroid to all points
    points = component_mesh.points
    distances = np.linalg.norm(points - centroid, axis=1)
    
    # Check the variance of the distances to identify a circular pattern
    variance = np.var(distances)
    threshold_variance = 0.1  # Set an appropriate threshold based on domain knowledge
    is_circular = variance < threshold_variance
    return is_circular

def classify_and_visualize_failure_types(mesh, components_meshes, displacement_vectors):
    """
    Classify and visualize the failure types for each component.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - components_meshes: List of PyVista mesh objects, each representing a connected failure surface.
    - displacement_vectors: NumPy array of displacement vectors for each cell in the mesh.
    """
    failure_types = []
    
    for i, component_mesh in enumerate(components_meshes):
        # Compute the centroid for the current component
        centroid = component_mesh.cell_centers().points.mean(axis=0)
        
        # Classify failure type
        failure_type = classify_failure_type(component_mesh, displacement_vectors, centroid)
        failure_types.append(failure_type)
        
        print(f"Component {i} classified as: {failure_type}")
    
    # Optional: Visualize classified failures
    plot_classified_failures(mesh, components_meshes, failure_types)

def plot_classified_failures(mesh, components_meshes, failure_types):
    """
    Visualize the compounds with their classified failure types.

    Parameters:
    - mesh: PyVista mesh object, the original mesh.
    - components_meshes: List of PyVista mesh objects, each representing a connected failure surface.
    - failure_types: List of strings indicating the type of failure for each component.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True, opacity=0.75)

    # Define colors for different failure types
    color_map = {
        'Planar Failure': 'blue',
        'Wedge Failure': 'green',
        'Toppling Failure': 'red',
        'Circular Failure': 'purple',
        'Compound Failure': 'yellow',
        'Step-Path Failure': 'cyan',
        'Multiple Wedge Failure': 'pink',
        'Rotational Failure': 'orange',
        'Irregular Failure': 'brown'
    }

    for i, (component_mesh, failure_type) in enumerate(zip(components_meshes, failure_types)):
        plotter.add_mesh(component_mesh, color=color_map[failure_type], show_edges=True, opacity=0.7)
    
    plotter.add_axes()
    plotter.add_title("Classified Failure Types of Compounds")
    plotter.show()

# Usage
displacement_magnitudes, displacement_vectors = calculate_displacement_magnitude_direction(mesh, U_global, failure_cells)
components_meshes = extract_connected_failure_surfaces(mesh, failure_cells)  # This function should already be defined in your code

# Classify and visualize failure types
classify_and_visualize_failure_types(mesh, components_meshes, displacement_vectors)


Calculating displacement magnitude and direction for failure cells...
Displacement calculation complete.
Extracting connected components of failure surfaces...
Found 6 connected components of failure surfaces.
Component 0 classified as: Rotational Failure
Component 1 classified as: Rotational Failure
Component 2 classified as: Rotational Failure
Component 3 classified as: Toppling Failure
Component 4 classified as: Toppling Failure
Component 5 classified as: Toppling Failure


Widget(value='<iframe src="http://localhost:61424/index.html?ui=P_0x255a79f5f10_12&reconnect=auto" class="pyvi…